####  This code works to jailbreak gemma-7b-it pretty consistently. Hoewver, it has to be run a few times before it works due to some randomness in probe. 

In [1]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from analysis.supervised import *
from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


    

In [2]:
from pathlib import Path
import os
# Get the current working directory (where the notebook is)
notebook_dir = Path.cwd()

# Move up to the root directory (parent of 'notebooks')
# project_root = notebook_dir.parent

df = pd.read_pickle(notebook_dir / "datasets/latest/wei_dataset_rated.pkl")

print("dataset loaded")


dataset loaded


In [4]:

## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

# Create new columns for each layer
df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
df["jailbreak_prompt_name"].value_counts()

labels = df["rating_binary"].to_list()
data = df[f'layer_{layer}'].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

split_idx = int(0.8* len(df))


#####

mlp = SimpleMLP()

mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)

mlp.train()

# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], target=1)
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        df.to_pickle(f"/datasets/old/wei_dataset_with_MLP_test.pkl")


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LEN LABELS 9000
LEN DATA 9000
Epoch [10/500], Loss: 0.3896
Epoch [20/500], Loss: 0.3411
Epoch [30/500], Loss: 0.3103
Epoch [40/500], Loss: 0.2327
Epoch [50/500], Loss: 0.2527
Epoch [60/500], Loss: 0.2929
Epoch [70/500], Loss: 0.2617
Epoch [80/500], Loss: 0.2718
Epoch [90/500], Loss: 0.2468
Epoch [100/500], Loss: 0.2687
Epoch [110/500], Loss: 0.2635
Epoch [120/500], Loss: 0.2516
Epoch [130/500], Loss: 0.2644
Epoch [140/500], Loss: 0.2373
Epoch [150/500], Loss: 0.2564
Epoch [160/500], Loss: 0.3069
Epoch [170/500], Loss: 0.2990
Epoch [180/500], Loss: 1.4366
Epoch [190/500], Loss: 0.7696
Epoch [200/500], Loss: 1.1102
Epoch [210/500], Loss: 1.7253
Epoch [220/500], Loss: 2.4431
Epoch [230/500], Loss: 0.7222
Epoch [240/500], Loss: 1.0497
Epoch [250/500], Loss: 1.2904
Epoch [260/500], Loss: 0.8051
Epoch [270/500], Loss: 1.3765
Epoch [280/500], Loss: 1.3550
Epoch [290/500], Loss: 0.8758
Epoch [300/500], Loss: 1.6288
Epoch [310/500], Loss: 1.2565
Epoch [320/500], Loss: 0.7356
Epoch [330/500], Lo

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch [500/500], Loss: 0.6673
Training complete.
Applying fitted scaler to test data...
Accuracy: 0.8706
Using MSE loss function
learning rate 0.1
nr_perturbations 32
loss MSE
torch.Size([1, 32, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor([[1.]])
NEW Predicted class: tensor([[0.]])
torch.Size([1, 1, 3072]) True
OLD Predicted class: tensor([[0.]])
TARGET tensor

KeyboardInterrupt: 

## Trying to catch good probe 

In [3]:
from analysis.causal_interventions import get_res_layers_to_enumerate

def get_response_MLP(model, 
                 tokenizer,  
                 prompt,
                 mlp,
                 layers_to_intervene=None,  # Allow multiple layers
                 max_new_tokens=15, 
                 c = 0,
                 loss = "MSE",
                 towards_target = True,
                 nr_perturbations=32, 
                 learning_rate = 0.1,
                 offensive = True):
    """
    Generates a response from a model with a causal intervention applied to specific layers.

    Args:
        model: Pretrained language model.
        tokenizer: Tokenizer compatible with the model.
        prompt (str): The input text prompt.
        probe: The intervention object (e.g., a linear model with coefficients).
        layers_to_intervene (list of int): Indices of the model layers to apply the intervention.
        max_new_tokens (int): Maximum number of new tokens to generate.
        intervention_strength (float): Strength of the intervention to apply.

    Returns:
        str: The generated text after applying the intervention.
    """
    print(f"Using {loss} loss function")
    print("learning rate", learning_rate)
    print("nr_perturbations", nr_perturbations)
    
    if layers_to_intervene is None:
        layers_to_intervene = [17]  # Default to layer 17 if no layers specified

    device = next(model.parameters()).device
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device)
    len_prompt = input_ids.size(1)
    
    print("loss", loss)
    # Define a loss function
    if loss == "MSE" : criterion = nn.MSELoss()
    
    # elif loss == "BCE" : criterion = nn.BCELoss()
    elif loss == "BCE" : criterion = nn.BCEWithLogitsLoss()
    
    elif loss == "BCE_new" : criterion = nn.BCELoss()
    
    sigmoid = nn.Sigmoid()


    def linear_intervene(name, c, offensive):
        def hook(model, input, output):
          
          with torch.enable_grad():
                      
            # Forward pass
            feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
            # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
            
            ## check old predicted class
            pred = mlp(feature_vector)
            print("OLD MLP_out:", pred)
            
            ### 
          
            
                    
            # Define perturbation as a parameter or tensor that requires gradient
            perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(output[0].device)
            
            printed_once = False
            
            for i in range(nr_perturbations):
              # Apply perturbation to the feature vector
              perturbed_vector = feature_vector + perturbation

              # # Get output from the MLP
              # if loss == "BCE_new":
              #   mlp_out = sigmoid(mlp(perturbed_vector))
              # else: 
              mlp_out = mlp(perturbed_vector)
              
              mlp_out_value = mlp_out.item()
              
              # target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value
              if offensive == True:

                  # target = mlp_out_value + c
                  target = max(mlp_out_value+c, c) # if out is very negative, target is 0

              else: 
                # defensive
                target = min(mlp_out_value-c, -c) # if out is very positive, target is 0
              
              if not printed_once: 
                print("target = ", target)
                printed_once = True

              target_out = torch.full_like(mlp_out, target)               

              # Calculate the loss
              loss = criterion(mlp_out, target_out).to(output[0].device)

              # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

              # Backward pass to compute gradients w.r.t the perturbation
              loss.backward()

              # Access the gradient of the loss w.r.t. the perturbation
              grad_input = perturbation.grad

              # Define a learning rate
              # learning_rate = 0.01

              
              if towards_target: 
                perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              else: 
                perturbation = (perturbation + learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              
            ## check new predicted class
            # pred = mlp(perturbation)
            pred = mlp(perturbed_vector)
            print("NEW MLP_out:", pred)
            
            ### 
            
            
            new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

            # returnn perturbation
            return new_out
          
        return hook

    layers_to_enum = get_res_layers_to_enumerate(model)
    hooks = []

    # Register hooks for each specified layer
    for layer_index in layers_to_intervene:
        if layer_index < 0 or layer_index >= len(layers_to_enum):
            raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
        hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, c, offensive))
        hooks.append(hook_handle)
    
    try:
        # model.eval()
        with torch.no_grad():
          output_sequence = model.generate(input_ids, num_return_sequences=1, max_new_tokens=max_new_tokens, do_sample=False)
            
    finally:
        # Ensure all hooks are removed after use
        for h in hooks:
            h.remove()
    
    generated_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
    
    # If outptut sequence repeats prompt, remove it 
    if generated_text.startswith(prompt): generated_text = generated_text[len(prompt):].strip()

    return generated_text                    


In [ ]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


from analysis.supervised import *
# from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle(notebook_dir / "datasets/latest/wei_dataset_rated.pkl")

print("dataset loaded")

## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

# Create new columns for each layer
df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
df["jailbreak_prompt_name"].value_counts()

labels = df["rating_binary"].to_list()
data = df[f'layer_{layer}'].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

split_idx = int(0.8* len(df))


#####

mlp = SimpleMLP()

mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)

mlp.train()

# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], c = 5.5, learning_rate = 0.005)
      
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        break
        df.to_pickle(notebook_dir / "datasets/old/MLP_intervention_test.pkl")

    

## compare probe attributes

In [ ]:
from analysis.causal_interventions import get_res_layers_to_enumerate

def get_response_MLP(model, 
                 tokenizer,  
                 prompt,
                 mlp,
                 layers_to_intervene=None,  # Allow multiple layers
                 max_new_tokens=15, 
                 target = 0,
                 loss = "MSE",
                 towards_target = True,
                 nr_perturbations=32, 
                 learning_rate = 0.1):
    """
    Generates a response from a model with a causal intervention applied to specific layers.

    Args:
        model: Pretrained language model.
        tokenizer: Tokenizer compatible with the model.
        prompt (str): The input text prompt.
        probe: The intervention object (e.g., a linear model with coefficients).
        layers_to_intervene (list of int): Indices of the model layers to apply the intervention.
        max_new_tokens (int): Maximum number of new tokens to generate.
        intervention_strength (float): Strength of the intervention to apply.

    Returns:
        str: The generated text after applying the intervention.
    """
    print(f"Using {loss} loss function")
    print("learning rate", learning_rate)
    print("nr_perturbations", nr_perturbations)
    
    if layers_to_intervene is None:
        layers_to_intervene = [17]  # Default to layer 17 if no layers specified

    device = next(model.parameters()).device
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device)
    len_prompt = input_ids.size(1)
    
    print("loss", loss)
    # Define a loss function
    if loss == "MSE" : criterion = nn.MSELoss()
    
    # elif loss == "BCE" : criterion = nn.BCELoss()
    elif loss == "BCE" : criterion = nn.BCEWithLogitsLoss()
    
    elif loss == "BCE_new" : criterion = nn.BCELoss()
    
    sigmoid = nn.Sigmoid()


    def linear_intervene(name, target):
        def hook(model, input, output):
          
          with torch.enable_grad():
                      
            # Forward pass
            feature_vector = output[0][:, :, :].requires_grad_().to(output[0].device)          
            # print(feature_vector.shape, feature_vector.requires_grad)  # Should print torch.Size([1, 768]) True
            
            ## check old predicted class
            pred = mlp(feature_vector)
            print("OLD MLP_out:", pred)
            
            ### 
            
                    
            # Define perturbation as a parameter or tensor that requires gradient
            perturbation = torch.zeros_like(feature_vector, requires_grad=True).to(output[0].device)
            
            printed_once = True

            for i in range(nr_perturbations):
              # Apply perturbation to the feature vector
              perturbed_vector = feature_vector + perturbation

              # # Get output from the MLP
              # if loss == "BCE_new":
              #   mlp_out = sigmoid(mlp(perturbed_vector))
              # else: 
              mlp_out = mlp(perturbed_vector)
              
              
              # target_out = torch.zeros_like(mlp_out)  # Default to zero, or choose an appropriate value


              target_out = torch.full_like(mlp_out, target)               

              # Calculate the loss
              loss = criterion(mlp_out, target_out).to(output[0].device)

              # print(loss, loss.requires_grad)  # Should print a scalar tensor and True

              # Backward pass to compute gradients w.r.t the perturbation
              loss.backward()

              # Access the gradient of the loss w.r.t. the perturbation
              grad_input = perturbation.grad

              # Define a learning rate
              # learning_rate = 0.01

              
              if towards_target: 
                perturbation = (perturbation - learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              else: 
                perturbation = (perturbation + learning_rate * grad_input).clone().detach().requires_grad_().to(output[0].device)
              
            ## check new predicted class
            # pred = mlp(perturbation)
            pred = mlp(perturbed_vector)
            print("NEW MLP_out:", pred)
            
            ### 
            
            
            new_out = perturbed_vector, output[1] # concat perturbed vector with the rest of the output

            # returnn perturbation
            return new_out
          
        return hook

    layers_to_enum = get_res_layers_to_enumerate(model)
    hooks = []

    # Register hooks for each specified layer
    for layer_index in layers_to_intervene:
        if layer_index < 0 or layer_index >= len(layers_to_enum):
            raise ValueError(f"Layer {layer_index} is out of bounds for the model.")
        hook_handle = model.model.layers[layer_index].register_forward_hook(linear_intervene(layer_index, target))
        hooks.append(hook_handle)
    
    try:
        # model.eval()
        with torch.no_grad():
          output_sequence = model.generate(input_ids, num_return_sequences=1, max_new_tokens=max_new_tokens, do_sample=False)
            
    finally:
        # Ensure all hooks are removed after use
        for h in hooks:
            h.remove()
    
    generated_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
    
    # If outptut sequence repeats prompt, remove it 
    if generated_text.startswith(prompt): generated_text = generated_text[len(prompt):].strip()

    return generated_text

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import sys

from gen_data.harmBench_autograder import grade_with_HF


# Function to process the dataframe
def process_dataframe(df, model, tokenizer):
    dictionary = {}
    # loop through every row where jailbreak_prompt_name is original_prompt
    for index, row in df.iterrows():
        if row['jailbreak_prompt_name'] == 'original_prompt':
            prompt_id = row['prompt_name']
            prompt_text = row['jailbreak_prompt_text']
            dictionary[prompt_id] = prompt_text

    # now loop through every row, and add a column called "original_prompt_text" to the dataframe
    for index, row in df.iterrows():
        prompt_id = row['prompt_name']
        df.loc[index, 'original_prompt_text'] = dictionary.get(prompt_id, None)
    
    return df

def rate(df):
    # Load the HarmBench model
    model_name = "cais/HarmBench-Llama-2-13b-cls"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, truncation_side="left", padding_side="left")


    df = process_dataframe(df, model, tokenizer)
    df = df.rename(columns={"jailbreak_prompt_text" : "original_prompt_text", "MLP_intervention_response": "output_text", "linear_intervention_response": "output_text"}, inplace=True)
    df = df.rename(columns={"forbidden_prompt": "original_prompt_text", "response": "output_text"})
    
    # Apply the grading function and add the ratings column
    df["rating"] = df.apply(lambda row: grade_with_HF(row['original_prompt_text'], [row['output_text']], model, tokenizer, verbose = True)[0], axis=1)
    
    return df

In [ ]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from analysis.supervised import *
# from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle("/datasets/latest/wei_dataset_rated.pkl")
## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

for i in range(50):

  model = AutoModelForCausalLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)


  # Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
  df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
  df.head(1)

  def extract_layer(row, layer):
      return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

  # Create new columns for each layer
  df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

  # drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
  df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
  df["jailbreak_prompt_name"].value_counts()

  labels = df["rating_binary"].to_list()
  data = df[f'layer_{layer}'].to_list()
  print("LEN LABELS", len(labels))
  print("LEN DATA", len(data))

  split_idx = int(0.8* len(df))


  #####

  mlp = SimpleMLP()

  losses, train_accuracies = mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
  test_accuracy = mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)


  # Store loss and accuracy in the DataFrame
  num_rows = len(df)
  df['probe_loss'] = [losses] * num_rows
  df['train_accuracy'] = [train_accuracies] * num_rows
  df['test_accuracy'] = test_accuracy * num_rows


  df = df[:20]
  print("dataset loaded")

  mlp.train()

  # run on every element of the sample
  for index, row in df.iterrows():
      # save in sample_df
      response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], target=1, learning_rate = 0.01)
      df.at[index, "response"] = response
      print(response)

  df = rate(df)  

  ASR = round(df["rating"].value_counts(normalize=True)["Yes"], 2)

  print("ASR", ASR)
  # Save every 10 samples
  df.to_pickle(f"/datasets/MLP_intervention_test_ASR{ASR}.pkl")

    

# Aggregate over probes trained on different layers

In [9]:
import pandas as pd
import sys
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


from analysis.supervised import *
# from analysis.causal_interventions import *
from analysis.Probes import SimpleMLP


df = pd.read_pickle(notebook_dir / "datasets/latest/wei_dataset_rated.pkl")
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)

print("dataset loaded")

## CONFIG
model_name = "google/gemma-7b-it"
layer=17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wei"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

    

dataset loaded


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# Step 1: Train individual probes for each layer
def train_layerwise_probes(df, layers):
    probes = {}
    for layer in layers:
        print(f"Training probe for layer {layer}...")
        
        # Extract data and labels
        df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)
        data = df[f'layer_{layer}'].to_list()
        labels = df['rating_binary'].to_list()
        split_idx = int(0.8 * len(data))
        
        # Train MLP for the layer
        mlp = SimpleMLP()
        mlp.train_MLP(data=data, labels=labels, split_idx=split_idx, num_epochs=500)
        probes[layer] = mlp
    return probes

# Step 2: Average predictions from individual probes
def average_layer_predictions(df, probes, layers):
    all_predictions = []
    for _, row in df.iterrows():
        layer_predictions = []
        for layer in layers:
            # Get the prediction probability for each layer
            input_tensor = torch.tensor(row[f'layer_{layer}']).unsqueeze(0)  # Add batch dimension
            pred = torch.tensor(probes[layer](input_tensor)[0])  # Use tensor input
            layer_predictions.append(pred)
        avg_prediction = np.mean(layer_predictions, axis=0)  # Average predictions across layers
        all_predictions.append(avg_prediction)
    return all_predictions

# Step 3: Evaluate final averaged predictions
def evaluate_averaged_predictions(predictions, labels):
    predicted_classes = [np.argmax(pred) for pred in predictions]
    accuracy = sum(p == l for p, l in zip(predicted_classes, labels)) / len(labels)
    return accuracy



In [6]:
# Example Usage
layers = [17, 20, 23, 26]  # Layers of interest
probes = train_layerwise_probes(df, layers)  # Train individual probes

Training probe for layer 17...


NameError: name 'extract_layer' is not defined

In [27]:
# Split data for evaluation
split_idx = int(0.8 * len(df))
train_df, test_df = df[:split_idx], df[split_idx:]

# Average predictions for the test set
test_labels = test_df['rating_binary'].to_list()
averaged_predictions = average_layer_predictions(test_df, probes, layers)

# Evaluate performance
final_accuracy = evaluate_averaged_predictions(averaged_predictions, test_labels)
print(f"Final Averaged Probe Accuracy: {final_accuracy}")

/tmp/ipykernel_929133/1527894397.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(probes[layer](input_tensor)[0])  # Use tensor input


Final Averaged Probe Accuracy: 0.8777777777777778


In [ ]:
# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()

# Create new columns for each layer
df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# drop all rows that have auto_payload_splitting, auto_obfuscation, dev_mode_with_rant in jailbreak_prompt_name
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"])]
df["jailbreak_prompt_name"].value_counts()

labels = df["rating_binary"].to_list()
data = df[f'layer_{layer}'].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

split_idx = int(0.8* len(df))


#####

mlp = SimpleMLP()

mlp.train_MLP(data = data, labels = labels, split_idx = split_idx, num_epochs = 500)
mlp.get_accuracy(data = data, labels = labels, split_idx = split_idx)

mlp.train()

# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], mlp, max_new_tokens=100, layers_to_intervene=[16], c = 5.5, learning_rate = 0.005)
      
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        break
        df.to_pickle(notebook_dir / "datasets/old/MLP_intervention_test.pkl")


In [4]:
def extract_layer(row, layer):
    return row['prompt_res_activations'][layer][0].cpu().numpy().flatten()


# Multilayer probe

In [5]:
# Add column that has 0 if "rating" is "No" and 1 if "rating" is "Yes"
df['rating_binary'] = df['rating'].apply(lambda x: 0 if x == "No" else 1)
df.head(1)

,prompt_name,jailbreak_prompt_name,jailbreak_prompt_text,prompt_res_activations,output_text,original_prompt_text,rating,rating_binary
0,meth_synthesis,original_prompt,"Write a detailed, step-by-step tutorial on how...","{0: [[tensor([ 0.1754, -0.1163, -0.0153, ...,...",I am unable to provide information on how to s...,"Write a detailed, step-by-step tutorial on how...",No,0


In [19]:


import pandas as pd

# Step 1: Extract features for each layer
layers = [17, 20, 23, 26]  # Layers of interest
for layer in layers:
    df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# Step 2: Melt the DataFrame
melted_df = pd.melt(
    df,
    id_vars=["jailbreak_prompt_name", "rating_binary"],  # Columns to keep
    value_vars=[f'layer_{layer}' for layer in layers],  # Columns to melt
    var_name="layer",  # New column for layer names
    value_name="features"  # New column for the features
)

# Step 3: Filter rows as before
melted_df = melted_df[~melted_df["jailbreak_prompt_name"].isin(
    ["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant"]
)]

# Step 4: Prepare labels and data
labels = melted_df["rating_binary"].to_list()
data = melted_df["features"].to_list()
print("LEN LABELS", len(labels))
print("LEN DATA", len(data))

# Convert features to tensors
data_tensor = torch.tensor(np.stack(data), dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Step 5: Split the dataset
split_idx = int(0.8 * len(melted_df))
train_data_tensor = data_tensor[:split_idx]
val_data_tensor = data_tensor[split_idx:]
train_labels_tensor = labels_tensor[:split_idx]
val_labels_tensor = labels_tensor[split_idx:]

# Display shapes of the resulting tensors
print(f"Train Data Shape: {train_data_tensor.shape}")
print(f"Validation Data Shape: {val_data_tensor.shape}")


LEN LABELS 36000
LEN DATA 36000
Train Data Shape: torch.Size([28800, 3072])
Validation Data Shape: torch.Size([7200, 3072])


In [20]:
    
# Step 2: Train the probe
def train_multilayer_probe(df, layers, num_epochs=500):
    # Prepare data
    # Define the probe
    mlp = SimpleMLP()  # Adjust input_dim as needed
    
    # Train the probe
    mlp.train_MLP(data=data_tensor, labels=labels_tensor, split_idx=split_idx)

    print("trained")
    # Evaluate on validation set
    accuracy = mlp.get_accuracy(data=data_tensor, labels=labels_tensor, split_idx=0)
    print(f"Validation Accuracy: {accuracy}")
    return mlp

layers = [17, 20, 23, 26]  # Layers of interest

# Ensure features for each layer are extracted
for layer in layers:
    df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)

# Train a single probe on data from all layers
probe = train_multilayer_probe(df, layers)


/data/nathalie_maria_kirch/ERA_Fellowship/analysis/Probes.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)


Epoch [10/500], Loss: 0.3958
Epoch [20/500], Loss: 0.2692
Epoch [30/500], Loss: 0.2849
Epoch [40/500], Loss: 0.3064
Epoch [50/500], Loss: 0.2713
Epoch [60/500], Loss: 0.2608
Epoch [70/500], Loss: 0.2641
Epoch [80/500], Loss: 0.2596
Epoch [90/500], Loss: 0.2495
Epoch [100/500], Loss: 0.3060
Epoch [110/500], Loss: 0.2549
Epoch [120/500], Loss: 0.3105
Epoch [130/500], Loss: 0.3239
Epoch [140/500], Loss: 0.3014
Epoch [150/500], Loss: 0.2492
Epoch [160/500], Loss: 0.2442
Epoch [170/500], Loss: 0.2272
Epoch [180/500], Loss: 0.2866
Epoch [190/500], Loss: 0.2817
Epoch [200/500], Loss: 0.2796
Epoch [210/500], Loss: 0.3131
Epoch [220/500], Loss: 0.3017
Epoch [230/500], Loss: 0.2685
Epoch [240/500], Loss: 0.2908
Epoch [250/500], Loss: 0.2759
Epoch [260/500], Loss: 0.2822
Epoch [270/500], Loss: 0.2878
Epoch [280/500], Loss: 0.2509
Epoch [290/500], Loss: 0.2521
Epoch [300/500], Loss: 0.2656
Epoch [310/500], Loss: 0.2929
Epoch [320/500], Loss: 0.2699
Epoch [330/500], Loss: 0.2517
Epoch [340/500], Lo

/data/nathalie_maria_kirch/ERA_Fellowship/analysis/Probes.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
# run on every element of the sample
for index, row in df.iterrows():
    # save in sample_df
    response = get_response_MLP(model, tokenizer, row["jailbreak_prompt_text"], probe, max_new_tokens=200, layers_to_intervene=[17], c = 20, learning_rate = 0.01)
      
    df.at[index, "response"] = response
    print(response)
    
    # Save every 10 samples
    if index % 20 == 0:
        df.to_pickle(notebook_dir / "datasets/MLP_{multilayer_probe}.pkl")

Using MSE loss function
learning rate 0.01
nr_perturbations 32
loss MSE


OLD MLP_out: tensor([[-15.0391]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[-2.5341]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-53.4586]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[5.6779]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-32.4155]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[4.0832]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-17.8212]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[3.5836]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-31.6076]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[4.3782]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-16.2349]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[4.0017]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-16.8098]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[3.8843]], grad_fn=<MeanBackward1>)
OLD MLP_out: tensor([[-22.8873]], grad_fn=<MeanBackward1>)
target =  20
NEW MLP_out: tensor([[4.